In [ ]:
from init_notebook import *
from src.algo.sdf.two_d.util import *
from src.algo.sdf.two_d import *

In [ ]:
def render(obj: Object, shape=(128, 128)):
    d = obj.render_distance(shape)
    inside = -d.clip(-0.01, .0) / .01
    stripes = (d * 10.) % 1. 
    stripes = np.pow(np.abs(stripes - .5) / .5, 4.)
    
    image = inside[..., None].repeat(3, 2) * .5
    image[..., 0] = stripes
    display(VF.to_pil_image(image))


In [ ]:
obj = Xor(
    #Plane((1, 0)).translate((.5, 0)),
    Circle(.3).translate((.5, .5)),
    Circle(.2).translate((.5, .8)),
    Circle(.1).translate((.7, .7)),
    Circle(.1).translate((.8, .7)),
    #radius=.2,
)#.invert()
render(obj)

In [ ]:

obj = Union(
    Circle(.5).scale(.5),
)
render(obj)

In [ ]:
render(
    Box((.3, .3)).rotate(45).subtract(
        Box((.2, .2)).subtract(
            Box((.1618, .03)), #smooth=.3
        ),
    ).translate((.5, .5))
)

In [ ]:
render(
    Box(.3).translate((.5, .5))
    .sin_warp(freq=4, amount=0.02, phase=(0, .5))
)

In [ ]:
render(
    Box(.3).translate((.5, .5))
    .noise_warp(freq=(1, 5), amount=0.1)
)
render(
    Box(.3).translate((.5, .5))
    .noise_warp_distance(freq=(5, 1), amount=0.1)
)

In [ ]:
m = (
    Box(0.2)
    #.sin_warp(amount=.008, freq=7, phase=(0.5, 0))
    .translate(.5)
    .render_mask(radius=.1, abs=True, shape=(384, 384))
)
VF.to_pil_image(m)

In [ ]:
rad = np.mgrid[:128, :128] / 128
#rad = rad[0] * .3
rad = (np.sin(rad[0]*11)+np.cos(rad[1]*17)) * .1# + .1
m = Box(.2).translate(.5).render_mask(radius=rad)
VF.to_pil_image(m)

In [ ]:
n = (
    Box(0.2)
    .noise_warp_distance(amount=1/300, freq=10)
    .noise_warp(amount=1/400, freq=30)
    #.sin_warp(amount=.008, freq=7, phase=(0.5, 0))
    .translate(.5)
    .render_mask_normal(
        z=100.3, radius=.1, abs=True, shape=(380, 380), 
        mask_func=lambda m: gain(m, 2)
        #mask_func=lambda m: m*m*(3-2*m)
        #mask_func=lambda m: np.pow(m, 10).clip(0, 1)
    )
)
n = calc_light(n, (-1, -1))
VF.to_pil_image(n * .5 + .5)

In [ ]:
n = calc_light(
    Union(
        Box(0.15),
        Circle(0.15).translate(.3),
    ).translate(.5)
    #.render_distance_normal()
    .render_mask_normal(radius=.2)
)
#n = n.repeat(2, -1)[..., 1:]
#n[..., 2] = 0
print(n.shape)
display(VF.to_pil_image(n*.5+.5))
#n

In [ ]:
class Tiles:
    L = 1
    T = 1 << 1
    R = 1 << 2
    B = 1 << 3

    TL = 1 << 4
    TR = 1 << 5
    BR = 1 << 6
    BL = 1 << 7

    ALL_EDGES = L|R|T|B
    ALL_CORNERS = TL|TR|BL|BR

    # 4x4
    EDGE_ORDER = [
        0, R, L|R, L,
        B, B|R, B|L|R, B|L,
        T|B, T|B|R, T|B|L|R, T|B|L,
        T, T|R, T|L|R, T|L,
    ]

    # 4x4
    CORNER_ORDER = [
        0, TR, TL|BR, BL,
        BR, BL|BR, BL|BR|TR, BL|TL,
        BL|TR, TL|TR|BR, TL|TR|BL|BR, TL|BL|BR,
        TL, TR|BR, TL|TR|BL, TL|TR, 
    ]

    # 7x7
    EDGE_AND_CORNER_ORDER = [
        TL|L|BL|T|B|TR|R|BR, TL|T|TR|L|BL|B|BR, TL|T|TR|BL, TL|T|TR, TL|T|TR|BR, TL|T|TR|R|BR|BL, TL|L|BL|T|B|TR|R|BR,
        BL|L|TL|T|TR|R|BR, BL|L|TL|T|TR|BR, TL|BL|BR, BL, TR|R|BR, TL|L|BL|TR|BR, TL|T|TR|R|BR|B|BL,
        TL|L|BL|TR, TL|TR|BR, BL|TL|TR|BR, TL|BL|B|BR, BL|TR, TL|TR, TL|T|TR|R|BR,  
        TL|L|BL, TR, TL|TR|R|BR, BL|L|TL|T|TR, TL, BR, BL|TR|R|BR,
        TL|L|BL|BR, BL|B|BR, BL|BR|TR, TL|BL, 0, TR|BR, TL|BL|TR|R|BR,
        TL|L|BL|B|BR|TR, TL|T|TR|BL|BR, TL|TR|BL, TL|BR, BL|BR, BL|B|BR|R|TR, TL|L|BL|TR|R|BR,
        TL|L|BL|T|B|TR|R|BR, TL|L|BL|B|BR|R|TR, TL|L|BL|B|BR, BL|B|BR|TR, TL|TR|BL|B|BR, TL|T|TR|BL|B|BR, TL|BL|B|BR|R|TR, 
    ]

    EO2 = [
        0, R, L|R|B|BR, L|R|BL|B|BR, L|R|BL|B, L|B, 0, 
        B, R|B, T|TR|L|R|B, TL|T|TR|L|R|B|BR, TL|T|L|BL|B, T|R|B, L, 
        T|R|B|BR, T|L|R|BL|B, T|L|R|B, T|TR|L|R, TL|T|L|R|B|BR, T|L|R|BL|B|BR, L|BL|B, 
        T|TR|R|B|BR, TL|T|L|R|BL|B|BR, T|L|BL|B, R|B|BR, T|TR|L|R|BL|B|BR, TL|T|TR|L|R|BL|B, TL|T|L|B, 
        T|TR|R|B, TL|T|TR|L|R, TL|T|L|R|B, T|TR|L|R|B|BR, TL|T|TR|L|R|BL|B|BR, TL|T|L|R|BL|B, T|L|B, 
        T|R, L|R|B, T|L|R|B|BR, T|TR|L|R|BL|B, TL|T|TR|L|R|B, TL|T|L, T|B, 
        0, T, T|TR|R, TL|T|L|R, T|L|R, L|R, T|L,
    ]

    @classmethod
    def edges(cls, index: int, radius: float = 0.1):
        if index & cls.ALL_EDGES == cls.ALL_EDGES:
            yield Box((2, 2))
        if index & cls.L:
            yield Box((1, radius)).translate((.5, 0))
        if index & cls.R:
            yield Box((1, radius)).translate((.5, 1))
        if index & cls.T:
            yield Box((radius, 1)).translate((0, .5))
        if index & cls.B:
            yield Box((radius, 1)).translate((1, .5))

    @classmethod
    def corners(cls, index: int, radius: float = 0.1):
        box = Circle(radius)
        if index & cls.TL:
            yield box
        if index & cls.TR:
            yield box.translate((0, 1))
        if index & cls.BL:
            yield box.translate((1, 0))
        if index & cls.BR:
            yield box.translate((1, 1))

    @classmethod
    def edge_circles(cls, index: int, radius: float = 0.1):
        if index & (cls.T|cls.L) == (cls.T|cls.L):
            yield Circle(1-radius).invert().translate((1, 1))
        if index & (cls.T|cls.R) == (cls.T|cls.R):
            yield Circle(1-radius).invert().translate((1, 0))
        if index & (cls.B|cls.L) == (cls.B|cls.L):
            yield Circle(1-radius).invert().translate((0, 1))
        if index & (cls.B|cls.R) == (cls.B|cls.R):
            yield Circle(1-radius).invert().translate((0, 0))

    @classmethod
    def edge_triangles(cls, index: int, radius: float=.25):
        o = Box(radius*np.sqrt(2)).rotate(45)
        io = Box((radius*.5, 2))
        if index & cls.ALL_EDGES == cls.ALL_EDGES:
            yield Box(.5).translate(.5)
        if index & cls.L:
            yield o.intersect(io.rotate(90)).translate((.5, 0))
        if index & cls.R:
            yield o.intersect(io.rotate(90)).translate((.5, 1))
        if index & cls.T:
            yield o.intersect(io).translate((0, .5))
        if index & cls.B:
            yield o.intersect(io).translate((1, .5))

    @classmethod
    def corner_triangles(cls, index: int, radius: float=.25):
        o = Box(radius*np.sqrt(2)).rotate(45)
        if index & cls.TL:
            yield o.translate((0, 0))
        if index & cls.TR:
            yield o.translate((0, 1))
        if index & cls.BL:
            yield o.translate((1, 0))
        if index & cls.BR:
            yield o.translate((1, 1))


def render_set(renderer, shape=(128, 128), padding=0):
    images = []
    for i in Tiles.EDGE_AND_CORNER_ORDER:
        image = renderer(i, shape=shape)
        images.append(VF.to_pil_image(image))
        
    return VF.to_pil_image(make_grid([VF.to_tensor(i) for i in images], nrow=7, padding=padding))

def renderer(index: int, shape=(128, 128)):
    obj = SmoothUnion(
        *Tiles.edges(index, radius=.2),
        *Tiles.corners(index, radius=.2),
        *Tiles.edge_circles(index, radius=.2),
        radius=.1,
    )#.sin_warp(freq=11, amount=.1/17)
    obj_n1 = obj.noise_warp(amount=1/100, freq=7)
    
    #mask = obj.round(.0).render_mask(shape=shape, abs=True, radius=.05)
    #norm1 = obj.render_mask_normal(shape=shape, abs=False, radius=.5)
    #norm2 = obj.round(.5).render_mask_normal(shape=shape, abs=True, radius=.6, z=10, mask_func=lambda m: gain(m, 5))
    #norm = mix(norm1, norm2[..., :2], 1)
    
    mask = step(calc_light(
        obj_n1.render_mask_normal(shape=shape, abs=False, radius=.2, z=10)
        , (-1, 0)), 0, -1)

    i1 = np.ones((*mask.shape, 3)) * np.array([[[.3, .8, .3]]])
    i2 = np.ones((*mask.shape, 3)) * np.array([[[.5, .4, .0]]])
    image = mix(i1, i2, mask)

    norm1 = obj_n1.render_mask_normal(
        shape=shape, abs=False, radius=.2, z=10,
        mask_func=lambda m: gain(m, 3)
    )
    light1 = calc_light(norm1, (0, -1))
    light2 = calc_light(
        obj_n1.render_mask_normal(
            shape=shape, abs=False, radius=.05, z=10,
            mask_func=lambda m: gain(m, 3)
        )
        , (-1, -.3)
    )
    light = mix(light1, light2, step(norm1[..., 0], 0, -1))[..., None]
    #mask = obj.render_mask(shape=shape)
    image += .5 * light.clip(0, 1)
    #image = mix(image, light * .5 + .5, .5)
    return image.clip(0, 1)
    
render_set(renderer, padding=1, shape=(64, 64))

In [ ]:
render_set(renderer, padding=0, shape=(16, 16))

In [ ]:

def water_noise(shape):
    img = np.ones((*shape, 3)) * np.array([[[.04, .2, .5]]])
    water = .3*perlin_noise_2d(shape, res=[5, 2], constant_edge=None, wrap=[True, True])
    water += .4*perlin_noise_2d(shape, res=[11, 4], constant_edge=None, wrap=[True, True])
    water += .13*perlin_noise_2d(shape, res=[23, 13], constant_edge=None, wrap=[True, True])
    waves = np.abs(perlin_noise_2d(shape, res=[5, 2], constant_edge=None, wrap=[True, True]))
    water += .2 * step(waves, .1, 0)
    
    return (.5 + .5*water).clip(0, 1) 

n = water_noise((128, 128))
VF.to_pil_image(make_grid(
    [torch.from_numpy(n).unsqueeze(0) for i in range(4*4)],
    nrow=4, padding=0,
))

In [ ]:
def water_tex(shape):
    img = np.ones((*shape, 3)) #* np.array([[[.1, .4, 1]]])
    water = water_noise(shape) * .5 + .3
    img[..., 0] *= np.pow(water, 3.)
    img[..., 1] *= np.pow(water, 2.)
    img[..., 2] *= np.pow(water, 1.)
    return img.clip(0, 1)

wtex1 = water_tex((128, 128))
VF.to_pil_image(make_grid(
    [torch.from_numpy(wtex1.transpose(2, 0, 1)) for i in range(4*4)],
    nrow=4, padding=0,
))

In [ ]:
#wtex1 = None
def renderer(index: int, shape=(128, 128)):
    global wtex1
    R = .4
    objs = [
        *Tiles.edges(index, radius=R),
        *Tiles.corners(index, radius=R),
        *Tiles.edge_circles(index, radius=R),
    ]
    obj = Union(*objs)
    obj_smooth = SmoothUnion(*objs, radius=.1)
    #.sin_warp(freq=11, amount=.1/17)
    obj_noise = (
        obj
        .noise_warp(amount=1/20, freq=3, constant_edge=.5)
        .noise_warp(amount=1/100, freq=7, constant_edge=0)
    )
    obj_noise_smooth = (
        obj_smooth
        .noise_warp(amount=1/30, freq=2, constant_edge=.5)
        #.noise_warp(amount=1/100, freq=7)
    )

    smooth_norm = obj_noise_smooth.render_mask_normal(
        shape=shape, abs=False, radius=.3, z=10,
        mask_func=lambda m: gain(m, 2)
    )
    top = step(smooth_norm[..., 0], 0, -1)
    bottom = step(smooth_norm[..., 0], 0, 1)
    left = step(smooth_norm[..., 1], 0, -1)
    right = step(smooth_norm[..., 1], 0, 1)

    mask_radius = .01 + .3*bottom# + .1*(left+right)
    smooth_norm2 = obj_smooth.render_mask_normal(
        shape=shape, abs=False, z=10,
        radius=mask_radius,
        mask_func=lambda m: gain(m, 2)
    )
    top2 = step(smooth_norm2[..., 0], 0, -1)
    bottom2 = step(smooth_norm2[..., 0], 0, 1)

    norm1 = obj_noise.round(-.03).render_mask_normal(
        shape=shape, abs=False, z=10,
        radius=mask_radius+0.04,
        mask_func=lambda m: gain(m, 3)
    )
    norm2 = obj_smooth.round(-.1).render_mask_normal(
        shape=shape, abs=False, z=10,
        radius=mask_radius+.13,
        mask_func=lambda m: gain(m, 5)
    )
    norm3 = obj_smooth.round(-.3).render_mask_normal(
        shape=shape, abs=False, z=10,
        radius=mask_radius+.6,
        mask_func=lambda m: gain(m, 5)
    )
    norm = mix(norm1, norm2, .5)
    norm = mix(norm, norm3, .5)
    light1 = calc_light(norm, (-.3, -1))
    
    i1 = np.ones((*light1.shape, 3)) * np.array([[[.3, .8, .3]]])
    i1 = i1 * (1. + .2 * perlin_noise_2d(shape=light1.shape, res=[11, 31]))[..., None]
    #i2 = np.ones((*light1.shape, 3)) * np.array([[[.04, .2, .5]]])
    if wtex1 is None:
        wtex1 = water_tex(light1.shape)
    i2 = wtex1
    i3 = np.ones((*light1.shape, 3)) * np.array([[[.5, .4, .0]]])

    image = mix(i1, i2, 1.-obj.render_mask(radius=mask_radius+.05, shape=shape))
    image = mix(image, i3, bottom2)

    #mask = obj.render_mask(shape=shape)
    image += .5 * light1.clip(-.5, 1)[..., None]
    #image = mix(image, light * .5 + .5, .5)
    return image.clip(0, 1)
    
render_set(renderer, padding=1)#, shape=(64, 64))

In [ ]:
render_set(renderer, padding=0, shape=(16, 16))

In [ ]:
VF.resize(render_set(renderer, padding=0, shape=(128, 128)), (16*7, 16*7), VF.InterpolationMode.BICUBIC)

In [ ]:

def renderer(index: int, shape=(128, 128)):
    global wtex1
    R = .05
    objs = [
        #*edge_triangles(index, radius=R),
        #*corner_triangles(index, radius=R/6),
        *Tiles.edges(index, radius=R),
        *Tiles.corners(index, radius=R),
        *Tiles.edge_circles(index, radius=R),
    ]
    obj = Union(*objs)
    
    mask = obj.round(0).render_mask(
        shape=shape, 
        radius=.6,
        #radius=.2, abs=True,
    )
    light = calc_light(
        obj.round(0).render_mask_normal(
            radius=.4, z=10, mask_func=lambda m: gain(m, 2),
        ),
        (-.3, -1)
    )
    image = (
        1-mask
        #(.3+.2*mask) 
        #* (.7+.2*light)
    )
    return image.clip(0, 1)
    
render_set(renderer, padding=1)#, shape=(64, 64))

In [ ]:
VF.resize(render_set(renderer, padding=0, shape=(128, 128)), (16*7, 16*7), VF.InterpolationMode.BICUBIC)

In [ ]:
def renderer(index: int, shape=(128, 128)):
    global wtex1
    R = .3
    objs = [
        *edge_triangles(index, radius=R),
        *corner_triangles(index, radius=R/6),
        #*Tiles.corners(index, radius=R),
        #*Tiles.edge_circles(index, radius=R),
    ]
    obj = Union(*objs)
    
    mask = obj.round(0).render_mask(
        shape=shape, 
        #radius=.5,
        radius=.2, abs=True,
    )
    light = calc_light(
        obj.round(.1).render_mask_normal(
            radius=.4, z=10, mask_func=lambda m: gain(m, 1.1),
        ),
        (.3, 1)
    )
    image = (
        #mask
        #(.3+.2*mask) 
        (.5+.5*light)
    )
    return image.clip(0, 1)
    
render_set(renderer, padding=1)#, shape=(64, 64))

In [ ]:
VF.resize(render_set(renderer, padding=0, shape=(128, 128)), (16*7, 16*7), VF.InterpolationMode.BICUBIC)